In [85]:
import pandas as pd
import numpy as np
import locale
locale.setlocale(locale.LC_ALL, '')
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing 
import re
import scipy as sp

plt.style.use('fivethirtyeight')

# 노트북 안에 그래프 그리기
%matplotlib inline
# 그래프 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일 사용
plt.style.use('ggplot')
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus']=False

# 컬럼을 다 보여주는 함수
from IPython.display import display
pd.options.display.max_columns = None

# 한글 font 설정
import matplotlib.font_manager as fm

font_name = fm.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
plt.rc('font',family = font_name)
mpl.rcParams['axes.unicode_minus'] = False

In [2]:
Product = pd.read_csv('Pruduct.csv')
print("Product")
Search1 = pd.read_csv('Search1.csv')
print("Search1")
Search2 = pd.read_csv('Search2.csv')
print("Search2")
Custom = pd.read_csv('Custom.csv')
print("Custom")
Master = pd.read_csv('Master.csv')
print("Master")
Session = pd.read_csv('Session.csv')
print("Session")

Product
Search1
Search2
Custom
Master
Session


In [5]:
print('Product의 shape',Product.shape)
print("-----------------------------")
print('Search1의 shape',Search1.shape)
print("-----------------------------")
print('Search2의 shape',Search2.shape)
print("-----------------------------")
print('Custom의 shape',Custom.shape)
print("-----------------------------")
print('Master의 shape',Master.shape)
print("-----------------------------")
print('Session의 shape',Session.shape)

Product의 shape (5024906, 8)
-----------------------------
Search1의 shape (2884943, 4)
-----------------------------
Search2의 shape (8051172, 3)
-----------------------------
Custom의 shape (671679, 3)
-----------------------------
Master의 shape (847652, 5)
-----------------------------
Session의 shape (2712907, 9)


# 1. 데이터 정제 및 탐색

## 1.1 Product

In [3]:
# 행동 한번으로 상품을 구매할 수 없다.
Product = Product[Product['HITS_SEQ'] != 1]

In [7]:
Product.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,4139680,7605037,12,642112,색상:워터멜론,[바비브라운],"39,000",1
1,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,데상트,"39,000",2
2,4142395,6158159,85,179538,(not set),[아베다],"39,000",1
3,4144914,7935714,12,554336,색상:블랙|사이즈:160cm(12~13세) / 1개,아디다스 키즈,"39,000",1
4,4144917,6406509,78,190306,5개,데코르테,"39,000",5


In [4]:
# PD_ADD_NM(추가정보) 열에서 | or / 로 세부정보를 구분
# |를 /로 변경하여 /로만 세부정보를 구분하게 하고 /로 세부정보를 분리하여 각각을 리스트로 저장
# 각 리스트 별로 양쪽에 공백자 제거
# 아직 사용 X ngram사용시 단어를 쪼개는데 어려움이 있음

def changeText1(text):
    return text.replace("|",' / ')

def changeText2(text):
    return text.split('/')

def stripText(text):
    a=[]
    for i in range(len(text)):
        a.append(text[i].strip())
    return a

In [5]:
Product['PD_ADD_NM'] = Product['PD_ADD_NM'].apply(changeText1)
# Product['PD_ADD_NM'] = Product['PD_ADD_NM'].apply(changeText2)
# Product['PD_ADD_NM'] = Product['PD_ADD_NM'].apply(stripText)

In [6]:
def cleanText(data):
    text = re.sub('[-=+,#/\?:^$.@*\"★※~&%ㆍ_!』\\‘|\(\)\[\]\<\>`\'…》]', '', data)
    return text

In [7]:
def cleanblank(data):
    text = re.sub(' ', '', data)
    return text

In [8]:
# 변수에 포함되어 있는 특수문자 제거

Product['PD_ADD_NM'] = Product['PD_ADD_NM'].apply(cleanText)
Product['PD_BRA_NM'] = Product['PD_BRA_NM'].apply(cleanText)
Product['PD_BUY_AM'] = Product['PD_BUY_AM'].apply(cleanText)

In [9]:
# 공백제거
Product['PD_ADD_NM'] = Product['PD_ADD_NM'].apply(cleanblank)

In [10]:
# PD_BUY_CT 의 1000이상의 값에서 , 발생
# 함수사용이 안됨 아마 , 가 있는데가 있고 없는데가 있어서..? 안되서 str로 강제변환후 , 를 공백으로 대체

Product["PD_BUY_CT"] = Product["PD_BUY_CT"].astype(str)
Product['PD_BUY_CT'] = Product['PD_BUY_CT'].str.replace(',','')
Product["PD_BUY_AM"] = Product["PD_BUY_AM"].astype(str)
Product['PD_BUY_AM'] = Product['PD_BUY_AM'].str.replace(',','')

# int로 형변환
Product['PD_BUY_AM'] = Product['PD_BUY_AM'].astype(int)
Product['PD_BUY_CT'] = Product['PD_BUY_CT'].astype(int)

## 1.2 Session

In [15]:
Session.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
0,5873599,8641867,9,20180509,82.0,"1,890",mobile,Daejeon,Daejeon
1,5873599,6616320,21,20180611,105.0,"1,604",mobile,Busan,Busan
2,5873599,5886172,40,20180624,41.0,632,mobile,Daejeon,Daejeon
3,5873884,1050889,15,20180913,160.0,"1,035",mobile,Gyeonggi-do,Anyang
4,5874461,10298270,5,20180412,13.0,298,mobile,Seoul,Seoul


In [11]:
# int형인 SESS_DT를 datetime형태로 형변환
Session['SESS_DT'] = pd.to_datetime(Session['SESS_DT'],format="%Y%m%d")

# year는 2018로 동일하므로 월 / 일 / 요일 변수를 새롭게 만듬
# SESS_DT는 병합에 사용될 수 있는 변수이므로 일단 제거 X
Session['SESS_MONTH'] = Session['SESS_DT'].dt.month
Session['SESS_DAY'] = Session['SESS_DT'].dt.day
Session['SESS_WEEK'] = Session['SESS_DT'].dt.weekday

In [12]:
# Session['TOT_SESS_HR_V']의 결측치 15000여개의 결측치 삭제
Session = Session[Session['TOT_SESS_HR_V'].notnull()]

# Session TOT_SESS_HR_V 의 형변환 (object -> int64)
Session['TOT_SESS_HR_V'] = Session['TOT_SESS_HR_V'].astype(str)
Session['TOT_SESS_HR_V'] = Session['TOT_SESS_HR_V'].str.replace(',','')
Session['TOT_SESS_HR_V'] = Session['TOT_SESS_HR_V'].astype(int)

In [13]:
# TOT_PAG_VIEW_CT 의 결측치 처리
# 예측변수에 사용하기위해 숫자로 매핑시킴
Session['DVC_CTG_NM'] = Session['DVC_CTG_NM'].map({'mobile':1,"desktop":2,'tablet':3})

In [14]:
# 총 9개의 결측치 존재 9개의 결측치는 삭제보다는 다른변수들을 이용하여 선형회귀예측값을 사용하는게 용이해 보임
from sklearn.linear_model import LinearRegression
Session_1 = Session[['SESS_SEQ','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','DVC_CTG_NM']]

linreg = LinearRegression()

x_train = Session_1[Session_1['TOT_PAG_VIEW_CT'].notnull()].drop(columns='TOT_PAG_VIEW_CT')
y_train = Session_1[Session_1['TOT_PAG_VIEW_CT'].notnull()]['TOT_PAG_VIEW_CT']
x_test = Session_1[Session_1['TOT_PAG_VIEW_CT'].isnull()].drop(columns='TOT_PAG_VIEW_CT')
y_test = Session_1[Session_1['TOT_PAG_VIEW_CT'].isnull()]['TOT_PAG_VIEW_CT']

linreg.fit(x_train, y_train)

predicted = linreg.predict(x_test)

Session.TOT_PAG_VIEW_CT[Session.TOT_PAG_VIEW_CT.isnull()] = predicted

In [15]:
# 얘만 float형이면 헷갈리니까 그냥 int형으로 변환
Session['TOT_PAG_VIEW_CT'] = Session['TOT_PAG_VIEW_CT'].astype(int)

In [16]:
# 예측변수에 사용하기위해 숫자로 매핑시킨걸 다시 문자로 매핑
# 다시 예측변수로 사용시 재매핑
Session['DVC_CTG_NM'] = Session['DVC_CTG_NM'].map({1:'mobile',2:"desktop",3:'tablet'})

In [17]:
Session.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,SESS_MONTH,SESS_DAY,SESS_WEEK
0,5873599,8641867,9,2018-05-09,82,1890,mobile,Daejeon,Daejeon,5,9,2
1,5873599,6616320,21,2018-06-11,105,1604,mobile,Busan,Busan,6,11,0
2,5873599,5886172,40,2018-06-24,41,632,mobile,Daejeon,Daejeon,6,24,6
3,5873884,1050889,15,2018-09-13,160,1035,mobile,Gyeonggi-do,Anyang,9,13,3
4,5874461,10298270,5,2018-04-12,13,298,mobile,Seoul,Seoul,4,12,3


## 1.3 Master

- 대분류 : 37개 / 중분류 : 128개 / 소분류 : 898 개

In [18]:
Master['PD_NM'] = Master['PD_NM'].apply(cleanText)
Master['PD_NM'] = Master['PD_NM'].apply(cleanblank)

In [19]:
Master.head()

,PD_C,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
0,64382,언더아머남성UAHG아머모크LS1289559001블랙MD95,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠
1,62282,여자플라워덧신2족선물세트174032set,속옷/양말/홈웨어,여성양말류,여성일반양말
2,61729,88A2933253배트맨스웨트티블랙130,유아동의류,유아의류상의,영유아티셔츠/탑
3,61537,닥터마틴아드리안블랙체리레드02체리레드250mm6,패션잡화,남성화,남성부츠
4,58820,여성그레이스트라이프퍼프소매블라우스128865YQ33회색앤틱실버S,남성의류,남성의류상의,남성남방셔츠


## 1.4 Custom

In [20]:
Custom.head()

,CLNT_ID,CLNT_GENDER,CLNT_AGE
0,4830726,F,30
1,4830874,F,40
2,4830975,F,30
3,4831275,F,30
4,4825325,F,30


In [21]:
# 30 - 40 - 20 - 50 대 ....

## 1.5 Search1

In [22]:
Search1.head()

,CLNT_ID,SESS_ID,KWD_NM,SEARCH_CNT
0,5607714,7112876,빌리프 아이크림,6
1,5607714,4090791,프리메라 마스크팩,3
2,5607714,4090791,여성청결제,1
3,5612428,1876482,명품가방,1
4,5612428,658123,콩순이 아이스크림,1


In [24]:
# 검색어에 특수문자 존재 할 수 있으므로 특수문자 모두 제거
Search1['KWD_NM'] = Search1['KWD_NM'].apply(cleanText)
Search1 = Search1[Search1['KWD_NM']!='']

Search1.reset_index(inplace=True)
Search1.drop('index',inplace=True,axis=1)

In [ ]:
Search1.reset_index(inplace=True)
Search1.drop('index',inplace=True,axis=1)

In [28]:
Search1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2884915 entries, 0 to 2884914
Data columns (total 4 columns):
CLNT_ID       int64
SESS_ID       int64
KWD_NM        object
SEARCH_CNT    int64
dtypes: int64(3), object(1)
memory usage: 88.0+ MB


## 1.6 Search2

In [25]:
Search2.head()

,SESS_DT,KWD_NM,SEARCH_CNT
0,20180407,닥스원피스,8
1,20180407,닥터지 브라이트닝,1
2,20180407,달팡 인트랄,2
3,20180407,대상트,1
4,20180407,더블유닷,6


In [26]:
# 검색어에 특수문자 존재 할 수 있으므로 특수문자 모두 제거
Search2['KWD_NM'] = Search2['KWD_NM'].apply(cleanText)
Search2 = Search2[Search2['KWD_NM']!='']

Search2['SEARCH_CNT'] = Search2['SEARCH_CNT'].apply(cleanText)
Search2['SEARCH_CNT'] = Search2['SEARCH_CNT'].astype(int)

Search2.reset_index(inplace=True)
Search2.drop('index',inplace=True,axis=1)

# 2. 데이터 병합
- 검색과 구매가 모두 이루어진 최종 테이블 생성

In [56]:
df_1 = pd.merge(Product,Master,on=["PD_C"],how='left')
df_1 = pd.merge(df_1,Custom,on=["CLNT_ID"],how='left')

In [57]:
# 회원 / 비회원 변수 생성
df_1['MEMBER'] = df_1['CLNT_GENDER'].isnull()
df_1['MEMBER'] = df_1['MEMBER'].map({True:0,False:1})

In [58]:
# Custom에 없는 구매상품정보가있음 -> 비회원주문

In [32]:
df_1.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_GENDER,CLNT_AGE,MEMBER
0,4139680,7605037,12,642112,색상워터멜론,바비브라운,39000,1,크러쉬드립칼라플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,NaN,NaN,0
1,4140076,10189797,13,570603,색상BLK0BLK0BLACK사이즈1202개,데상트,39000,2,데상트스포츠베이직빅로고티셔츠DMGRDMELANGE115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,F,40.0,1
2,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,NaN,NaN,0
3,4144914,7935714,12,554336,색상블랙사이즈160cm1213세1개,아디다스 키즈,39000,1,YGGU34TGTABQ2873블랙110cm6세,유아동의류,여아의류하의,여아레깅스,M,40.0,1
4,4144917,6406509,78,190306,5개,데코르테,39000,5,단독허니듀세트5만원상당샘플증정,화장품/뷰티케어,스킨케어,스킨케어세트,NaN,NaN,0


In [59]:
# GENDER와 AGE는 비회원일 경우 Nan이기 때문에 소분류 를 통해 최빈값으로 대체
# 비회원이기 때문에 정보가 부족하기 때문에 완벽한 대체가 어려움
# 따라서 구매한 상품의 소분류의 각각의 최빈값으로 결측값을 채움

# 최빈값 추출 함수
def mode(x): return sp.stats.mode(x)[0][0]

In [60]:
####  GENDER
data = df_1[['CLAC3_NM','CLNT_GENDER']]

# 각각의 소분류 별로 최빈값을 추출
data_2 = df_1.groupby('CLAC3_NM')['CLNT_GENDER'].agg(mode)

# data와 data_2를 병합시켜 하나의 데이터프레임으로 만들어 줌
data_3 = pd.merge(data,data_2,on=['CLAC3_NM'],how='left')

# 원래값이 Nan이 아닌 애들은 그대로 그값을 사용, Nan값인 애들은 최빈값으로 대체 해줌
data_3['CLNT_GENDER'] = np.where(pd.notnull(data_3['CLNT_GENDER_x'])==True,data_3['CLNT_GENDER_x'],data_3['CLNT_GENDER_y'])

# Nan이 없는 새로운 값들로 열 대체
df_1['CLNT_GENDER'] = data_3['CLNT_GENDER']
df_1.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_GENDER,CLNT_AGE,MEMBER
0,4139680,7605037,12,642112,색상워터멜론,바비브라운,39000,1,크러쉬드립칼라플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,F,NaN,0
1,4140076,10189797,13,570603,색상BLK0BLK0BLACK사이즈1202개,데상트,39000,2,데상트스포츠베이직빅로고티셔츠DMGRDMELANGE115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,F,40.0,1
2,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,NaN,0
3,4144914,7935714,12,554336,색상블랙사이즈160cm1213세1개,아디다스 키즈,39000,1,YGGU34TGTABQ2873블랙110cm6세,유아동의류,여아의류하의,여아레깅스,M,40.0,1
4,4144917,6406509,78,190306,5개,데코르테,39000,5,단독허니듀세트5만원상당샘플증정,화장품/뷰티케어,스킨케어,스킨케어세트,F,NaN,0


In [61]:
####  AGE
data = df_1[['CLAC3_NM','CLNT_AGE']]

# 각각의 소분류 별로 최빈값을 추출
data_2 = df_1.groupby('CLAC3_NM')['CLNT_AGE'].agg(mode)

# data와 data_2를 병합시켜 하나의 데이터프레임으로 만들어 줌
data_3 = pd.merge(data,data_2,on=['CLAC3_NM'],how='left')

# 원래값이 Nan이 아닌 애들은 그대로 그값을 사용, Nan값인 애들은 최빈값으로 대체 해줌
data_3['CLNT_AGE'] = np.where(pd.notnull(data_3['CLNT_AGE_x'])==True,data_3['CLNT_AGE_x'],data_3['CLNT_AGE_y'])

# Nan이 없는 새로운 값들로 열 대체
df_1['CLNT_AGE'] = data_3['CLNT_AGE']
df_1.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_GENDER,CLNT_AGE,MEMBER
0,4139680,7605037,12,642112,색상워터멜론,바비브라운,39000,1,크러쉬드립칼라플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,F,30.0,0
1,4140076,10189797,13,570603,색상BLK0BLK0BLACK사이즈1202개,데상트,39000,2,데상트스포츠베이직빅로고티셔츠DMGRDMELANGE115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,F,40.0,1
2,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0
3,4144914,7935714,12,554336,색상블랙사이즈160cm1213세1개,아디다스 키즈,39000,1,YGGU34TGTABQ2873블랙110cm6세,유아동의류,여아의류하의,여아레깅스,M,40.0,1
4,4144917,6406509,78,190306,5개,데코르테,39000,5,단독허니듀세트5만원상당샘플증정,화장품/뷰티케어,스킨케어,스킨케어세트,F,30.0,0


In [ ]:
# 여기서부터는 Session과 Search1 과Search2를병합하는 과정

In [ ]:
# Session + Search1
# Session과 Search1을 병합하여 Search2를 사용하기위한 테이블을 생성
# Session + Search1 전처리
search_data = pd.merge(Session,Search1,on=['CLNT_ID','SESS_ID'],how='left')
search_data = search_data[search_data['SEARCH_CNT'].notnull()]
# search_data = search_data[['CLNT_ID','SESS_ID','SESS_DT','KWD_NM','SEARCH_CNT']]

In [50]:
search_data['SESS_DT'] = search_data['SESS_DT'].astype(str)
search_data['SESS_DT'] = search_data['SESS_DT'].apply(cleanText)
search_data['SESS_DT'] = search_data['SESS_DT'].astype(int)

search_data['SEARCH_CNT'] = search_data['SEARCH_CNT'].astype(int)

In [51]:
# Search2와 병합
Search = pd.merge(search_data,Search2,on=['SESS_DT','KWD_NM'],how='left')

In [ ]:
# Search 테이블 생성
# 목적성에 중심을 둔다면 두 변수 모두 변형없이 그대로 사용해도 괜찮을듯
# Search1의 SEARCH_CNT는 내가 얼마나 고민했는가의 정도(사려고 혹은 마음에 들어서)
# Search2의 SEARCH_CNT는 이 사람들이 해당 검색어(상품)을 얼마나 검색헀는지(인기의 척도)

In [52]:
Search.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,SESS_MONTH,SESS_DAY,SESS_WEEK,KWD_NM,SEARCH_CNT_x,SEARCH_CNT_y
0,5873599,8641867,9,20180509,82,1890,mobile,Daejeon,Daejeon,5,9,2,닥스손수건세트,1,10
1,5873599,6616320,21,20180611,105,1604,mobile,Busan,Busan,6,11,0,조르지오아르마니 쿠션,3,19
2,5873599,5886172,40,20180624,41,632,mobile,Daejeon,Daejeon,6,24,6,맥 틴트,2,12
3,5873884,1050889,15,20180913,160,1035,mobile,Gyeonggi-do,Anyang,9,13,3,칠부내의,1,9
4,5873884,1050889,15,20180913,160,1035,mobile,Gyeonggi-do,Anyang,9,13,3,아동팬티,1,14


In [88]:
# 세션 정보가 없는 구매 데이터는 제외하고 진행(약 1.7만개)
# Session + Search1 + Search2합치고 더 진행
df_1 = pd.merge(df_1,Search,on=["CLNT_ID","SESS_ID"],how='inner')
df_1.head()

KeyboardInterrupt: 

In [64]:
# Session + Search1 + Search2합치고 더 진행

In [65]:
df_1.shape

(6442939, 28)

In [42]:
################################ 병합 끝 ####################################

In [43]:
## 수정
# PD_ADD_NM, PD_BRA_NM, PD_NM, CLAC1_NM, CLAC2_NM, CLAC3_NM
# vs KWD_NM
# 모두 동일하게 진행하여 검색지수를 생성
# 검색지수 : 고객이 얼마나 목적성을 갖고 검색을 하는가에 대한 지표

#################### 검색과정 ####################

# 대/중/소 분류로 검색을 하는 고객, 특정 브랜드로 검색을 하는 고객, 상품명, (사이즈, 색상)
# 등으로 다양하게 검색이 이루어질것으로 예상
# 6개의 변수와 Search1의 검색어변수의 유사도를 구하여 실제로 구매로 이루어지는 정도(가중치)를
# 파악하여 가중합으로 검색지표를 생성

# 문제점 
# 반팔 <> 티셔츠 => 같은 의미의 단어지만 검색을 어떻게 했느냐에따라 검색지수가 다르게 나옴
# 문장유사도 분석으로는 이런차이를 모두 확인하기 어려움

In [71]:
data = df_1[['CLNT_ID','SESS_ID','PD_C','PD_ADD_NM','PD_BRA_NM','PD_NM','CLAC1_NM','CLAC2_NM','CLAC3_NM','KWD_NM']]

In [72]:
data.head()

,CLNT_ID,SESS_ID,PD_C,PD_ADD_NM,PD_BRA_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,KWD_NM
0,4142395,6158159,179538,notset,아베다,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,입생로랑
1,4142395,6158159,179538,notset,아베다,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,이솝
2,4142395,6158159,179538,notset,아베다,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,아베다 스무드
3,4142395,6158159,179538,notset,아베다,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,아베다
4,4142395,6158159,558739,상품명이솝제라늄리프바디클렌저,이솝,이솝제라늄리프바디클렌저500ml이솝제라늄리프바디클렌저,퍼스널케어,바디케어,바디워시,입생로랑


In [73]:
# 문장의 유사도 분석
# (KWD_NM과 PD_BRA_NM의 공통 단어) / (KWD_NM의 단어)
# 단어를 2음절씩 나눈이유 : 2음절씩 나누면 문장의 단어를 최대한 세분화해서볼수있음
# 1개는 단어의 의미를 잃기 때문에 안됨

## n_gram
def ngram(s,num):
    res = []
    slen = len(s)#-num+1
    for i in range(slen):
        ss = s[i:i+num]
        res.append(ss)
    return res

# 문장 유사도
def diff_ngram(sa,sb,num):
    a = ngram(sa,num)
    b = ngram(sb,num)
    r = []
    cnt = 0
    for i in a:
        for j in b:
            if i==j:
                cnt +=1
                r.append(i)
    return cnt / len(b)#, r
# 겹치는 단어가 어떤건지 필요하면 r 도 함께 return

# 변수로 생성
def search_index(df,a):
    r = []
    for i in range(len(df)):
        rr = diff_ngram(df[a].iloc[i],df["KWD_NM"].iloc[i],2)
        r.append(rr)
    return r

In [75]:
# 컬럼별 문장 유사도 생성 
data['brand_search_index'] = search_index(data,"PD_BRA_NM")
print('brand')
data['PD_NM_search_index'] = search_index(data,"PD_NM")
print('PD_NM')
data['PD_ADD_NM_search_index'] = search_index(data,"PD_ADD_NM")
print('PD_ADD')
data['CLAC1_NM_search_index'] = search_index(data,"CLAC1_NM")
print('CLAC1')
data['CLAC2_NM_search_index'] = search_index(data,"CLAC2_NM")
print('CLAC2')
data['CLAC3_NM_search_index'] = search_index(data,"CLAC3_NM")
print("CLAC3")

brand
PD_NM
PD_ADD
CLAC1
CLAC2
CLAC3


In [ ]:
# PD_NM과 KWD_NM과의 ngram 분석시 1이넘는 값들이 존재
# 같은 단어가 반복해서 나오면 분모보다 분자가 큰 경우가 발생
# P(A|B) -> 조건부 확률 : B가 일어났을때 A가 일어날 확률 
# = P(AΠB) / P(B) : 검색(B)이 이루어졌을때 해당 상품정보로 구매(A)할 확률
# 이 값들은 어짜피 0보다 크면 어떠한 목적성이 있다고 판단하여 값을 1로 할 것이기 때문에 문제X

In [76]:
# 최종 변수 생성
data['index'] = data['brand_search_index']+data['PD_NM_search_index']+data['PD_ADD_NM_search_index']+data['CLAC1_NM_search_index']+data['CLAC2_NM_search_index']+data['CLAC3_NM_search_index']

In [77]:
data.head()

,CLNT_ID,SESS_ID,PD_C,PD_ADD_NM,PD_BRA_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,KWD_NM,brand_search_index,PD_NM_search_index,PD_ADD_NM_search_index,CLAC1_NM_search_index,CLAC2_NM_search_index,CLAC3_NM_search_index,index
0,4142395,6158159,179538,notset,아베다,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,입생로랑,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,4142395,6158159,179538,notset,아베다,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,이솝,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,4142395,6158159,179538,notset,아베다,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,아베다 스무드,0.285714,0.0,0.0,0.0,0.0,0.0,0.285714
3,4142395,6158159,179538,notset,아베다,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,아베다,1.000000,0.0,0.0,0.0,0.0,0.0,1.000000
4,4142395,6158159,558739,상품명이솝제라늄리프바디클렌저,이솝,이솝제라늄리프바디클렌저500ml이솝제라늄리프바디클렌저,퍼스널케어,바디케어,바디워시,입생로랑,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [78]:
# 검색 목적 지수를 binary 형태로 변환 하여 이진분류로 만들어 줌
data["search_purpose"] = np.where(data["index"] == 0, 0, 1)

In [81]:
# Product , Master , Custom, Session, Search1 이 합쳐진 테이블에 타겟변수 생성
df_1['search_purpose'] = data['search_purpose']
df_1['search_purpose_index'] = data['index']

In [83]:
df_1.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_GENDER,CLNT_AGE,MEMBER,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,SESS_MONTH,SESS_DAY,SESS_WEEK,KWD_NM,SEARCH_CNT_x,SEARCH_CNT_y,search_purpose,search_purpose_index
0,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,20180619,107,6524,desktop,Seoul,Seoul,6,19,1,입생로랑,3,427,0,0.000000
1,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,20180619,107,6524,desktop,Seoul,Seoul,6,19,1,이솝,3,51,0,0.000000
2,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,20180619,107,6524,desktop,Seoul,Seoul,6,19,1,아베다 스무드,1,1,1,0.285714
3,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,20180619,107,6524,desktop,Seoul,Seoul,6,19,1,아베다,23,110,1,1.000000
4,4142395,6158159,23,558739,상품명이솝제라늄리프바디클렌저,이솝,56000,1,이솝제라늄리프바디클렌저500ml이솝제라늄리프바디클렌저,퍼스널케어,바디케어,바디워시,F,30.0,0,1,20180619,107,6524,desktop,Seoul,Seoul,6,19,1,입생로랑,3,427,0,0.000000


In [84]:
# df_1.to_csv("final_data.csv",index=False)